In [4]:
import pandas as pd
import numpy as np
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import joblib
import pickle

# Download stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Load CSV file (your uploaded file)
df = pd.read_csv("/content/r5fskJhv - my-trello-board.csv")

# Basic inspection
df.head()


,Card ID,Card Name,Card URL,Card Description,Labels,Members,Due Date,Attachment Count,Attachment Links,Checklist Item Total Count,...,Vote Count,Comment Count,Last Activity Date,List ID,List Name,Board ID,Board Name,Archived,Start Date,Due Complete
0,655cdca037c6e0c35990ebd2,Project planning,https://trello.com/c/DCgisR4d/2-project-planning,Log performance metrics after implementation.,(red),NaN,2025-09-22T00:00:00.000Z,0,NaN,0,...,0,0,2023-11-21T16:36:48.263Z,655cdc9f9793094c8793c60d,To do,655cdc9eabd978ea86d620ba,My Trello board,False,NaN,False
1,655cdca097b19d34cead1519,Kickoff meeting,https://trello.com/c/8fS8CSS9/1-kickoff-meeting,Test thoroughly before deployment.,(yellow),NaN,2025-07-10T00:00:00.000Z,0,NaN,0,...,0,0,2023-11-21T16:36:48.158Z,655cdc9f9793094c8793c60d,To do,655cdc9eabd978ea86d620ba,My Trello board,False,NaN,False
2,686a2dd6e7cc85dcf3615607,Write backend API documentation,https://trello.com/c/qEo1F4Vv/4-write-backend-...,Generate and write Swagger docs for the new AP...,(red),NaN,2025-07-12T00:00:00.000Z,0,NaN,0,...,0,0,2025-07-06T08:03:34.607Z,655cdc9f9793094c8793c60d,To do,655cdc9eabd978ea86d620ba,My Trello board,False,NaN,False
3,686a312e3e789f3efd3db653,Implement cache layer #7,https://trello.com/c/Rh1oumxN/11-implement-cac...,Coordinate with the team to resolve this issue.,(yellow),NaN,2025-07-30T00:00:00.000Z,0,NaN,0,...,0,0,2025-07-06T08:17:50.807Z,655cdc9f9793094c8793c60d,To do,655cdc9eabd978ea86d620ba,My Trello board,False,NaN,False
4,686a312fed4b4dd930819b83,Update dashboard UI #10,https://trello.com/c/FbKVuXlm/14-update-dashbo...,Coordinate with the team to resolve this issue.,(yellow),NaN,2025-09-09T00:00:00.000Z,0,NaN,0,...,0,0,2025-07-06T08:17:52.034Z,655cdc9f9793094c8793c60d,To do,655cdc9eabd978ea86d620ba,My Trello board,False,NaN,False


In [6]:
# Define mapping from label colors to priority levels
priority_map = {
    "(red)": "High",
    "(yellow)": "Medium",
    "(green)": "Low"
}

# Create new 'Priority' column
df['Priority'] = df['Labels'].map(priority_map)

# Drop rows where priority couldn't be determined
df = df.dropna(subset=['Priority'])


In [7]:
# Merge Card Name and Description for text analysis
df['text'] = df['Card Name'].fillna('') + ' ' + df['Card Description'].fillna('')


In [8]:
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered = [word for word in words if word not in stop_words]
    return " ".join(filtered)

# Apply cleaning
df['clean_text'] = df['text'].apply(clean_text)


In [9]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['clean_text'])  # Feature set


In [10]:
le = LabelEncoder()
y = le.fit_transform(df['Priority'])  # Target variable


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [12]:
model = MultinomialNB()
model.fit(X_train, y_train)


MultinomialNB()

In [13]:
y_pred = model.predict(X_test)

print("📊 Accuracy:", accuracy_score(y_test, y_pred))
print("🧾 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("📄 Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))


📊 Accuracy: 0.30198019801980197
🧾 Confusion Matrix:
 [[16 25 34]
 [14 15 31]
 [22 15 30]]
📄 Classification Report:
               precision    recall  f1-score   support

        High       0.31      0.21      0.25        75
         Low       0.27      0.25      0.26        60
      Medium       0.32      0.45      0.37        67

    accuracy                           0.30       202
   macro avg       0.30      0.30      0.29       202
weighted avg       0.30      0.30      0.29       202



In [14]:
# Save as Pickle
with open("priority_predictor.pkl", "wb") as f:
    pickle.dump(model, f)

# Save as Joblib
joblib.dump(model, "priority_predictor.joblib")

# Save the TF-IDF vectorizer and label encoder too
joblib.dump(tfidf, "tfidf_vectorizer.joblib")
joblib.dump(le, "label_encoder.joblib")


['label_encoder.joblib']

In [15]:
def predict_priority(task_text):
    cleaned = clean_text(task_text)
    vect_text = tfidf.transform([cleaned])
    pred = model.predict(vect_text)
    return le.inverse_transform(pred)[0]

# Try it
sample_task = "Fix urgent bug in production"
print("Predicted Priority:", predict_priority(sample_task))


Predicted Priority: Low


In [18]:
!pip install streamlit

import streamlit as st
import joblib
import string
import nltk
from nltk.corpus import stopwords

# Ensure stopwords are available
nltk.download('stopwords')

# Load saved models
model = joblib.load("/content/priority_predictor.joblib")
tfidf = joblib.load("/content/tfidf_vectorizer.joblib")
le = joblib.load("/content/label_encoder.joblib")

# Clean text function
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered = [word for word in words if word not in stop_words]
    return " ".join(filtered)

# Streamlit UI
st.set_page_config(page_title="AI Task Priority Predictor", layout="centered")
st.title("🧠 AI Task Priority Predictor")
st.write("Enter your task details and get a predicted **priority level** based on AI.")

# Input box
task_input = st.text_area("✍️ Describe your task here", height=150)

# Predict button
if st.button("🔍 Predict Priority"):
    if not task_input.strip():
        st.warning("Please enter some task text.")
    else:
        cleaned_text = clean_text(task_input)
        vect_text = tfidf.transform([cleaned_text])
        prediction = model.predict(vect_text)
        priority = le.inverse_transform(prediction)[0]

        # Show result
        st.success(f"🧩 Predicted Priority: **{priority.upper()}**")

        # Optional: color display
        if priority == "High":
            st.markdown("🔴 This task is **High Priority**. Handle it urgently!", unsafe_allow_html=True)
        elif priority == "Medium":
            st.markdown("🟡 This task is **Medium Priority**. Handle it soon.", unsafe_allow_html=True)
        else:
            st.markdown("🟢 This task is **Low Priority**. You can handle it later.", unsafe_allow_html=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025-07-28 06:15:05.777 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-28 06:15:05.778 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-28 06:15:05.885 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-07-28 06:15:05.886 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-28 06:15:05.888 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-28 06:15:05.890 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in 